# A. EXTRACT

In [2]:
# Import package yang dibutuhkan
from selenium import webdriver  # untuk membuka browser secara otomatis 
from selenium.webdriver.common.by import By     # untuk mencari elemen di halaman browser dengan CSS Selector
from bs4 import BeautifulSoup   # untuk menganalisis HTML yang dimuat selenium menjadi objek yang mudah dicari
import pandas as pd, time   # import pandas untuk menyimpan hasil scrapping dalam dataframe, time untuk jeda agar produk ter load dengan baik

In [3]:
# Membuat dataframe kosong
df = pd.DataFrame()

# list kosong untuk penampung sementara masing masing kolom
brands = []
names  = []
genders = []
colors  = []
prices  = []

# Membuka browser
driver = webdriver.Chrome()

# Halaman yang akan di scrapping
URL = "https://www.footlocker.id/new-arrivals.html"

# Fungsi untuk auto scroll 
def scroll_until_loaded(css_item="li.product-item, div.product-item",max_loops=20, wait_s=2, patience=4):     # max loops: batas maksimum scroll agar tidak loop terus menerus, wait_s: memberikan jeda setelah scroll, patience: sebagai batas ketika item tidak bertambah akan dianggap selesai
    """"
    Melakukan auto scroll pada halaman dengan menggunakan infinite-scroll hingga semua produk ter load

    Cara Kerja:
    1. menghitung jumlah elemen produk (cur) berdasarkan selector css_item
    2. membandingkan junmlah saat ini (cur) dengan jumlah sebelumnya (last):
        - jika cur > last: ada produk baru yang masuk, update last = cur dan reset counter (same = 0)
        - jika cur = last: tidak ada penambahan 
    3. jika same >= patience: tidak bertambah maka dianggap semua produk sudah termuat
    4. jika masih ada produk baru:
        - scroll ke elemen terakhir agar memuat lambat 
        - lanjut scroll ke bawah dengan scrollBy(0, 400)
    5. memberi jeda untuk memberi waktu halaman memuat
    """     
    same = 0
    last = 0
    for _ in range(max_loops):
        items = driver.find_elements(By.CSS_SELECTOR, css_item)     
        cur = len(items)
        if cur > last:
            last = cur
            same = 0
        else:
            same += 1
            if same >= patience:
                break
        if items:
            driver.execute_script("arguments[0].scrollIntoView({block:'end'});", items[-1])
        driver.execute_script("window.scrollBy(0, 400);")
        time.sleep(wait_s)

try:
    driver.get(URL)     # membuka halaman
    time.sleep(2)   # memberi jeda agar produk tampil

    scroll_until_loaded()   # scroll hingga semua produk tampil semua

    page = BeautifulSoup(driver.page_source, "html.parser")     # ekstrak html dari halaman
    cards = page.select("li.product-item, div.product-item")    #menagmbil semua produk

    for c in cards:
        # menemukan setiap elemen 
        brand  = c.select_one("h3.product-brand span")
        name   = c.select_one("h3.product.name.product-item-name a.product-item-link")
        gender = c.select_one("div.gender")
        color  = c.select_one("div.color span.value")
        price  = c.select_one("span.price")

        # extract setiap elemen
        brands.append(brand.get_text(strip=True) if brand else None)
        names.append(name.get_text(strip=True) if name else None)
        genders.append(gender.get_text(strip=True) if gender else None)
        colors.append(color.get_text(strip=True) if color else None)
        prices.append(price.get_text(strip=True) if price else None)

finally:
    driver.quit()   # tutup browser

# semua list disimpan disini sesuai list yang dibuat diatas
df["brand"] = brands
df["nama_produk"] = names
df["gender"] = genders
df["warna"] = colors
df["harga"] = prices

# Menampilkan 10 produk teratas
print(df.head(10))


         brand                                        nama_produk  \
0         Nike          Air Max Plus Men's Sneakers Shoes - Black   
1         Nike                P-6000 Men's Sneakers Shoes - White   
2        ASICS  FX GEL-1130 Unisex Sneakers Shoes - Swamp Gree...   
3         Nike  Vomero 5 Boys' Grade School Sneakers Shoes - P...   
4         Nike  P-6000 Men's Sneakers Shoes - Lt Orewood Brn/L...   
5       ADIDAS               Samba OG Men's Sneakers - Core Black   
6  NEW BALANCE               740 Boys Sneakers Shoes - Light Grey   
7  NEW BALANCE                    530 Boys Sneakers Shoes - Beige   
8         Nike  P-6000 Boys' Grade School Sneakers Shoes - Sum...   
9     CONVERSE  Run Star Trainer Unisex Sneakers - Black/Egret...   

           gender    warna          harga  
0            Pria    Hitam  Rp. 2.389.000  
1            Pria    Putih  Rp. 1.429.000  
2          Unisex    Hijau  Rp. 1.799.000  
3  Anak Laki-Laki   Netral  Rp. 1.499.000  
4            Pria   

In [36]:
# Menyimpan file
df.to_csv("footlocker_new-arrivals.csv", index=False)


# B. TRANSFORM

### Eksplorasi Data Sederhana

In [ ]:
# menampilkan 10 produk teratas
df.head(10)

,brand,nama_produk,gender,warna,harga
0,Nike,Air Max Plus Men's Sneakers Shoes - Black,Pria,Hitam,2389000
1,Nike,P-6000 Men's Sneakers Shoes - White,Pria,Putih,1429000
2,ASICS,FX GEL-1130 Unisex Sneakers Shoes - Swamp Gree...,Unisex,Hijau,1799000
3,Nike,Vomero 5 Boys' Grade School Sneakers Shoes - P...,Anak Laki-Laki,Netral,1499000
4,Nike,P-6000 Men's Sneakers Shoes - Lt Orewood Brn/L...,Pria,Coklat,1429000
5,ADIDAS,Samba OG Men's Sneakers - Core Black,Pria,Hitam,2200000
6,NEW BALANCE,740 Boys Sneakers Shoes - Light Grey,Anak Laki-Laki,Abu-abu,1199000
7,NEW BALANCE,530 Boys Sneakers Shoes - Beige,Anak Laki-Laki,Netral,1099000
8,Nike,P-6000 Boys' Grade School Sneakers Shoes - Sum...,Anak Laki-Laki,Putih,1329000
9,CONVERSE,Run Star Trainer Unisex Sneakers - Black/Egret...,Unisex,Hitam,1299000


In [ ]:
# untuk melihat apakah ada nilai NaN dalam dataframe
df.isnull().sum()

brand          0
nama_produk    0
gender         0
warna          0
harga          0
dtype: int64 



In [ ]:
# untuk membersihkan data harga supaya bisa diproses sebagai angka
# karena harga masih bertype object maka akan diganti int64 sesuai dengan type nya
df["harga"] = (df["harga"].str.replace(r"[^\d]", "", regex=True).replace({"": pd.NA, "None": pd.NA}))
df["harga"] = df["harga"].astype("Int64")


In [ ]:
# untuk melihat info semua data setelah harga dirubah type menjadi int
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 461 entries, 0 to 460
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   brand        461 non-null    object
 1   nama_produk  461 non-null    object
 2   gender       461 non-null    object
 3   warna        461 non-null    object
 4   harga        461 non-null    Int64 
dtypes: Int64(1), object(4)
memory usage: 18.6+ KB


In [28]:
# menyimpan data setelah di transform
df.to_csv("footlocker_new-arrivals_transform.csv", index=False)
